# Imports and Setup

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as T
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
import albumentations as A
import os
import random
import gc

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold

def clear_memory():
  torch.cuda.empty_cache()
  gc.collect()

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount("/content/dirve", force_remount=True)

os.chdir("/content/dirve/MyDrive/Colab Notebooks/poster/specs_eegs_workspace")

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(20)

# Initiate Constants

In [ ]:
TRAIN_CSV = "../train.csv"
FULL_SPECTROGRAMS = "./specs.npy"
FULL_EEG = "./eeg_specs.npy"

GROUP_FOLDS = 5

LABELS = pd.read_csv(TRAIN_CSV).columns[-6:].to_list()

# Metadata Preprocessing
- group by eeg_id
- add min/max offset of spectrogram offset seconds in each spectrogram_id
- take the mean of votes
- add total evaluator votes
- add kl_loss comparing to 1/6 uniform distribution

In [ ]:
# metadata preprocessing
train_df = pd.read_csv(TRAIN_CSV)

LABELS = ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
KEEP_COLS = ["spectrogram_id", "spectrogram_label_offset_seconds", "patient_id", "expert_consensus"]


min_temp = train_df.groupby("eeg_id")[["spectrogram_label_offset_seconds"]].agg({
    "spectrogram_label_offset_seconds": "min"
    }).reset_index()

min_temp.columns = ["eeg_id", "min"]

max_temp = train_df.groupby("eeg_id")[["spectrogram_label_offset_seconds"]].agg({
    "spectrogram_label_offset_seconds": "max"
    }).reset_index()

max_temp.columns = ["eeg_id", "max"]


temp = pd.merge(min_temp, max_temp, on="eeg_id", how="left")

del min_temp, max_temp

# group by eeg_id
train_df = train_df.groupby("eeg_id")[KEEP_COLS + LABELS].agg(
    {**{m: "first" for m in KEEP_COLS},
    **{t: "sum" for t in LABELS}}
    ).reset_index()

# count total evaluators
train_df["num_evaluators"] = train_df[LABELS].values.sum(axis=1)

# take mean of labels
train_df[LABELS] = train_df[LABELS]/train_df[LABELS].values.sum(axis=1, keepdims=True)

# add group fold
gkf = GroupKFold(n_splits=GROUP_FOLDS)
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, train_df[LABELS], train_df["patient_id"])):
    train_df.loc[val_idx, "fold"] = int(fold+1)
train_df.head()

# compute kl_loss
votes = train_df[LABELS].values + 1e-6

# add kl_loss
train_df["kl_loss"] = F.kl_div(
    torch.log(torch.tensor(votes)),
    torch.tensor([1/6]*6),
    reduction="none"
    ).sum(dim=1).numpy()

# merge
train_df = pd.merge(train_df, temp, on="eeg_id", how="left")

# Setup GPU and Load Train Data

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read in full spectrograms and eeg data
print("Reading in all spectrograms and eegs data...")
full_eegs = np.load(FULL_EEG, allow_pickle=True).item()
full_specs = np.load(FULL_SPECTROGRAMS, allow_pickle=True).item()
print("Reading in all spectrograms and eegs data Complete!")

# Define Custom Dataset
- shape: N x 126 x 256 x 8 channels for X
- 0-3 channels are kaggle spectrograms, 4-7 are EEG to spectrograms
- data augmentation: horizontal flip with p=0.5


In [ ]:
class CustomDataset(Dataset):
  def __init__(self, metadata, specs_dict, eegs_dict, transforms=False, mode="Train"):
    self.metadata = metadata
    self.specs_dict = specs_dict
    self.eegs_dict = eegs_dict
    self.transforms = transforms
    self.mode = mode
    self.__epsilon = 1e-6

  def __len__(self):
    return len(self.metadata)

  def __getitem__(self, index):
    X, y = self.__create_data(index)

    if self.transforms:
        X = self.__data_transformation(X)

    # convert to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

  def __create_data(self, index):
    X = np.zeros((128, 256, 8), dtype=np.float32)
    y = np.zeros((6), dtype=np.float32)
    img = np.ones((128, 256), dtype=np.float32)
    row = self.metadata.iloc[index]

    if self.mode == "Train":
        # offset = int((row["min"] + row["max"])//4)
        offset = int(row["spectrogram_label_offset_seconds"] // 2)
        # convert labels to float32
        y = row[LABELS].values.astype(np.float32)

    else:
        offset = 0

    for i in range(4):
        # 100x300xi -> i: LL(0), RL(1), LP(2), RP(3)
        img = self.specs_dict[row["spectrogram_id"]][offset:offset+300, i*100:(i+1)*100].T

        # log transform
        # img = np.clip(img, np.exp(-4), np.exp(8))
        img = np.clip(img, np.exp(-6), np.exp(10))
        img = np.log(img)

        # standardization
        m = np.nanmean(img.flatten())
        std = np.nanstd(img.flatten())
        img = (img - m) / (std + self.__epsilon)
        # img = np.nan_to_num(img, 0.0)
        img = np.nan_to_num(img, -1)

        # fit into X (0-3 at dim=3)
        # 100x256xi
        # X[14: -14, :, i] = img[:, 22:-22] / 2.0
        X[14: -14, :, i] = img[:, 22:-22]

    # eegs
    # 128x256x4 LL(4), RL(5), LP(6), RP(7)
    img = self.eegs_dict[row["eeg_id"]]
    X[:, :, 4:] = img

    return X, y


  def __data_transformation(self, x):
    transforms = A.Compose([A.HorizontalFlip(p=0.5)])
    return transforms(image=x)["image"]

# Define Transfer Learning Models
- with reshaping methods
1. reshaping from (N, 128, 256, 8) to (N, 512, 512, 3)
2. reshaping from (N, 128, 256, 8) to (N, 256, 256, 12)

In [ ]:
class Effnet_512(nn.Module):
  def __init__(self, num_classes=6, pretrained=True):
    super().__init__()
    self.model = models.efficientnet_b2(pretrained=pretrained)
    num_features = self.model.classifier[1].in_features
    self.model.classifier = torch.nn.Linear(num_features, num_classes)

  def forward(self, x):
    x = self.__reshape_input(x)
    x = self.model(x)
    return x

  def __reshape_input(self, x):
    """
    reshape input from (N, 128, 256, 8) to (N, 512, 512, 3) and permute to (N, 3, 512, 512)
    """
    spectrograms = [x[:, :, :, i:i+1] for i in range(4)]
    spectrograms = torch.cat(spectrograms, dim=1)
    eegs = [x[:, :, :, i:i+1] for i in range(4, 8)]
    eegs = torch.cat(eegs, dim=1)
    # now spectrograms and eegs are both Nx512x256x1
    x = torch.cat([spectrograms, eegs], dim=2)
    x = torch.cat([x, x, x], dim=3)
    x = x.permute(0, 3, 1, 2)
    return x

class Effnet_256(nn.Module):
    def __init__(self, num_classes=6, pretrained=True):
        super().__init__()
        self.model = models.efficientnet_b2(pretrained=pretrained)
        self.model.features[0][0] = nn.Conv2d(12, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        num_features = self.model.classifier[1].in_features
        self.model.classifier = torch.nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.__reshape_input(x)
        x = self.model(x)
        return x

    def __reshape_input(self, x):
        """
        (N, 128, 256, 8) -> (N, 256, 256, 12])
        """
        # LL + RL 256 x 256
        spectrograms_1 = [x[:, :, :, i:i+1] for i in range(2)]
        spectrograms_1 = torch.cat(spectrograms_1, dim=1)

        # LP + RP 256 x 256
        spectrograms_2 = [x[:, :, :, i:i+1] for i in range(2, 4)]
        spectrograms_2 = torch.cat(spectrograms_2, dim=1)

        # LL + LP 256 x 256
        eegs_1 = [x[:, :, :, i:i+1] for i in range(4, 6)]
        eegs_1 = torch.cat(eegs_1, dim=1)

        # LP + RP 256 x 256
        eegs_2 = [x[:, :, :, i:i+1] for i in range(6, 8)]
        eegs_2 = torch.cat(eegs_2, dim=1)

        # stack
        x = torch.cat([spectrograms_1, spectrograms_2, eegs_1, eegs_2], dim=3)
        x = torch.cat([x, x, x], dim=3)

        # permute
        x = x.permute(0, 3, 1, 2)
        return x

# Define Train and Val Functions

In [ ]:
def train(train_loader, model, loss_function, optimizer, epoch, num_epoch, device, iteration):
  losses = []
  model.train()
  num_iters = 0
  for x, y in train_loader:
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      scores = F.log_softmax(scores, dim=1)
      loss = loss_function(scores, y)
      if num_iters % iteration == 0:
          print(f"Epoch {epoch+1}/{num_epoch} at {num_iters}, Loss: {loss.item():.4f}")
      losses.append(loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      num_iters += 1

  return np.mean(losses)

def valid(val_loader, model, loss_function, device):
  model.eval()
  predictions_dict = {}
  predictions = []
  losses = []
  with torch.no_grad():
    for x, y in val_loader:
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      scores = F.log_softmax(scores, dim=1)
      loss = loss_function(scores, y)
      losses.append(loss.item())
      y_pred = F.softmax(scores, dim=1)
      predictions.append(y_pred.to("cpu").numpy())
  predictions_dict["predictions"] = np.concatenate(predictions)
  return np.mean(losses), predictions_dict

# Define Train Model Function
- in order to switch models

In [ ]:
def train_model(model_name:str, model_class, naming:str, size:int, total_votes=False):

  BATCH_SIZE = 8

  STAGE_ONE_EPOCH = 5
  STAGE_ONE_LR = 1e-4

  STAGE_TWO_EPOCH = 3
  STAGE_TWO_LR = 1e-5

  ITERATION = 100

  for fold in range(GROUP_FOLDS):
    print(f"### Fold {fold+1} Start Training...")
    train_folds = train_df[train_df["fold"] != fold+1]
    val_folds = train_df[train_df["fold"] == fold+1]

    train_dataset = CustomDataset(train_folds, full_specs, full_eegs, transforms=False, mode="Train")
    val_dataset = CustomDataset(val_folds, full_specs, full_eegs, transforms=False, mode="Train")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
    print(f"# Train size: {len(train_loader)}")

    # model
    model = model_class()
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=STAGE_ONE_LR)
    kl_loss = nn.KLDivLoss(reduction="batchmean")

    # train and val
    best_loss = np.inf
    for epoch in range(STAGE_ONE_EPOCH):
      # train
      avg_train_loss = train(train_loader, model, kl_loss, optimizer, epoch, STAGE_ONE_EPOCH, device, ITERATION)
      # val
      avg_val_loss, prediction_dict = valid(val_loader, model, kl_loss, device)
      predictions = prediction_dict["predictions"]

      if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save({"model": model.state_dict(),
                    "predictions": predictions},
                   os.path.join("./models", f"{model_name}_{naming}_{size}_fold_{fold+1}_best.pth"))
      clear_memory()


    # stage 2 training
    print(f"### Fold {fold+1} Start Training Stage 2...")
    if total_votes:
      train_stage2 = train_df[(train_df["fold"] != fold+1) & (train_df["num_evaluators"] >= 10)]
      val_stage2 = train_df[(train_df["fold"] == fold+1) & (train_df["num_evaluators"] >= 10)]
    else:
      train_stage2 = train_df[(train_df["fold"] != fold+1) & (train_df["kl_loss"] < 5.5)]
      val_stage2 = train_df[(train_df["fold"] == fold+1) & (train_df["kl_loss"] < 5.5)]

    train_dataset_stage2 = CustomDataset(train_stage2, full_specs, full_eegs, transforms=False, mode="Train")
    val_dataset_stage2 = CustomDataset(val_stage2, full_specs, full_eegs, transforms=False, mode="Train")

    train_loader_stage2 = DataLoader(train_dataset_stage2, batch_size=BATCH_SIZE, shuffle=True)
    val_loader_stage2 = DataLoader(val_dataset_stage2, batch_size=BATCH_SIZE, shuffle=True)

    print(f"# Train size: {len(train_loader_stage2)}")

    best_model_path = os.path.join("./models", f"{model_name}_{naming}_{size}_fold_{fold+1}_best.pth")
    saved_model = torch.load(best_model_path)
    model.load_state_dict(saved_model["model"])

    best_loss = np.inf
    optimizer = optim.Adam(model.parameters(), lr=STAGE_TWO_LR)
    for epoch in range(STAGE_TWO_EPOCH):
      # train
      avg_train_loss = train(train_loader_stage2, model, kl_loss, optimizer, epoch, STAGE_TWO_EPOCH, device, ITERATION)
      # val
      avg_val_loss, prediction_dict = valid(val_loader, model, kl_loss, device)
      predictions = prediction_dict["predictions"]

      if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save({"model": model.state_dict(),
                    "predictions": predictions},
                   os.path.join("./models", f"{model_name}_{naming}_{size}_fold_{fold+1}_best.pth"))
      clear_memory()

In [ ]:
def main():
  train_model("EfficientNet_b2", Effnet_256, "2stage_kl_loss", 256, total_votes=False)
  train_model("EfficientNet_b2", Effnet_512, "2stage_kl_loss", 512, total_votes=False)
if __name__ == "__main__":
    main()